In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
from utilities import LogManager

### Without using config.yaml

If config_file is undefined, class default parameters will be loaded.

The following is acceptable.

In [47]:
lm = LogManager()

Copy enabled (default behavior)
Config file not provided, initializing logger with class default config.
Signal handlers registered


Get LogManager's config

In [48]:
lm.config

{'formats': {'simple': '{time:YYYY-MM-DD HH:mm:ss} | {level: <8} | {extra[logger_name]} | {file: <16} | {function} : {line} - {message}',
  'console': ' <green>{time:YYYY-MM-DD HH:mm:ss}</green> | <level>{level: <8}</level> | <red>{extra[logger_name]}</red> | <cyan>{file: <16} | {function}</cyan> : {line} - <white>{message}</white>'},
 'handlers': {'handler_file': {'sink': './logs/.logs',
   'format': '{time:YYYY-MM-DD HH:mm:ss} | {level: <8} | {extra[logger_name]} | {file: <16} | {function} : {line} - {message}',
   'level': 'DEBUG',
   'filter': <function utilities.logger._logging_manager.LoggingManager._make_handler_filter.<locals>.filter_func(record)>},
  'handler_console': {'sink': <ipykernel.iostream.OutStream at 0x21567499f60>,
   'format': ' <green>{time:YYYY-MM-DD HH:mm:ss}</green> | <level>{level: <8}</level> | <red>{extra[logger_name]}</red> | <cyan>{file: <16} | {function}</cyan> : {line} - <white>{message}</white>',
   'level': 'DEBUG',
   'filter': <function utilities.log

Example logging calls

In [49]:
# Note: if LogManager(), the default logger_name is "default_task".
logger_main = lm.get_logger("default_task")
logger_main.debug("some debug message")
logger_main.info("some info message")
logger_main.warning("some warning message")
logger_main.error("some error message")
logger_main.success("some success message")

 2025-09-03 22:19:10 | DEBUG    | default_task | 4292057848.py    | <module> : 3 - some debug message
 2025-09-03 22:19:10 | INFO     | default_task | 4292057848.py    | <module> : 4 - some info message
 2025-09-03 22:19:10 | WARNING  | default_task | 4292057848.py    | <module> : 5 - some warning message
 2025-09-03 22:19:10 | ERROR    | default_task | 4292057848.py    | <module> : 6 - some error message
 2025-09-03 22:19:10 | SUCCESS  | default_task | 4292057848.py    | <module> : 7 - some success message


The following is also acceptable.

In [ ]:
lm = LogManager(timezone = "Antarctica/South_Pole")

Example logging calls:

In [ ]:
logger_main = lm.get_logger("default_task")
logger_main.debug("some debug message")
logger_main.info("some info message")
logger_main.warning("some warning message")
logger_main.error("some error message")
logger_main.success("some success message")

#### Note: The following will raise an assertion error:

In [ ]:
logger_main = lm.get_logger("new_task")

Run `lm.add_logger` before getting logger for this new task

### Using config.yaml

In [6]:
config_file = "./logger_config.yaml"
lm = LogManager(config_file)

Copy enabled (default behavior)
Signal handlers registered


Get Loggers

In [7]:
logger_A = lm.get_logger("logger_a")
logger_B = lm.get_logger("logger_b")

In [8]:
# Example use
logger_A.info("should only appear in console")
logger_A.critical("should appear in both console and file")
logger_B.debug("this should not appear since level for handler_console config is set to INFO")
logger_B.critical("should also appear in console, but for another task")

 2025-09-03 21:34:09 | INFO     | logger_a | 3623765976.py    | <module> : 2 - should only appear in console
 2025-09-03 21:34:09 | CRITICAL | logger_a | 3623765976.py    | <module> : 3 - should appear in both console and file
 2025-09-03 21:34:09 | CRITICAL | logger_b | 3623765976.py    | <module> : 5 - should also appear in console, but for another task


Assertion error is raised if an undefined logger is called.

In [ ]:
logger_c = lm.get_logger("logger_c")  # this should raise an assertion error, logger does not exist

Add new handler and logger

The following format (simple) is accepted as long as it's defined in config file under 'formats'

In [10]:
lm.add_handler(
    "handler_console_simple",
    {
        "sink": "sys.stdout",
        "format": "simple",
        "level": "info"
    }
)

Note that error will not be raised if format is not found in the config; only warning will be shown.

In [11]:
lm.add_handler(
    "handler_xxx",
    {
        "sink": "sys.stdout",
        "format": "simpleeee",
        "level": "info"
    }
)

 ⚠️ The format referenced by handler 'handler_xxx' is not defined in the 'formats' section of the config file. Using the format as is: 
	 simpleeee 



Because it may be a valid custom format:

In [12]:
lm.add_handler(
    "handler_console_fire",
    {
        "sink": "sys.stdout",
        "format": "🔥{time:YYYY-MM-DD HH:mm:ss} | {level: <8} | {extra[logger_name]} | {file: <16} | {function} : {line} - {message}",
        "level": "info",
    }
)

 ⚠️ The format referenced by handler 'handler_console_fire' is not defined in the 'formats' section of the config file. Using the format as is: 
	 🔥{time:YYYY-MM-DD HH:mm:ss} | {level: <8} | {extra[logger_name]} | {file: <16} | {function} : {line} - {message} 



In [13]:
lm.add_logger("logger_c", [{"handler": "handler_console_fire", "level": "DEBUG"}])

In [14]:
logger_C = lm.get_logger("logger_c")
# Note: the effective logging level is the higher of the handler's and logger's levels.
# Therefore, DEBUG messages will not be logged because the handler's level is set to INFO.

logger_C.debug("this should not print since handler_console_fire is set to INFO")  # Will not be logged
logger_C.info("this should print")   # Will be logged
logger_C.error("this should print too")

🔥2025-09-03 21:39:50 | INFO     | logger_c | 2748549182.py    | <module> : 6 - this should print
🔥2025-09-03 21:39:50 | ERROR    | logger_c | 2748549182.py    | <module> : 7 - this should print too


Update handler

In [15]:
lm.update_handler(
    "handler_console_fire",
    {
        "sink": "sys.stdout",
        "format": "🧯{time:YYYY-MM-DD HH:mm:ss} | {level: <8} | {extra[logger_name]} | {file: <16} | {function} : {line} - {message}",
        "level": "debug",
    }
)

 ⚠️ The format referenced by handler 'handler_console_fire' is not defined in the 'formats' section of the config file. Using the format as is: 
	 🧯{time:YYYY-MM-DD HH:mm:ss} | {level: <8} | {extra[logger_name]} | {file: <16} | {function} : {line} - {message} 



In [16]:
logger_C.debug("this should print now since handler and logger config is set to DEBUG")  # Will be logged
logger_C.info("this should print")
logger_C.error("this should print too")

🧯2025-09-03 21:41:01 | DEBUG    | logger_c | 1071904596.py    | <module> : 1 - this should print now since handler and logger config is set to DEBUG
🧯2025-09-03 21:41:01 | INFO     | logger_c | 1071904596.py    | <module> : 2 - this should print
🧯2025-09-03 21:41:01 | ERROR    | logger_c | 1071904596.py    | <module> : 3 - this should print too


Update logger

In [18]:
lm.update_logger("logger_c", [{"handler": "handler_console_fire", "level": "ERROR"}, {"handler": "handler_console", "level": "ERROR"}])

In [19]:
logger_C.debug("this should print since both logger config is set to ERROR")
logger_C.error("this should print twice for logger_c, 1 for handler_console_fire and the other from newly added handler_console")

 2025-09-03 21:42:38 | ERROR    | logger_c | 3130521273.py    | <module> : 2 - this should print twice for logger_c, 1 for handler_console_fire and the other from newly added handler_console
🧯2025-09-03 21:42:38 | ERROR    | logger_c | 3130521273.py    | <module> : 2 - this should print twice for logger_c, 1 for handler_console_fire and the other from newly added handler_console


Remove logger

In [20]:
lm.remove_logger("logger_c")
logger_C.debug("this should NOT appear on the console")
logger_C.info("this should NOT appear on the console")
logger_C.error("this should NOT appear on the console")
logger_A.info("only this should appear on the console")

 2025-09-03 21:43:46 | INFO     | logger_a | 2861226126.py    | <module> : 5 - only this should appear on the console


Remove handler

In [21]:
lm.remove_handler("handler_console")
logger_A.debug("this should NOT appear on the FILE.")
logger_A.info("this should NOT appear on the FILE.")
logger_A.error("this should appear on the FILE.")

### Copying log files

In [50]:
lm = LogManager()

Copy enabled (default behavior)
Config file not provided, initializing logger with class default config.
Signal handlers registered


In [ ]:
lm.add_handler(
    "handler_copy_test_1",
    {
        "sink": "./logs/copy_test_1.log",
        "format": "{time} {level} {message}",
        "level": "DEBUG"
    }
)

lm.add_handler(
    "handler_copy_test_2",
    {
        "sink": "./logs/subfolder/copy_test_2.log",
        "format": "{level} {message}",
        "level": "DEBUG"
    }
)

lm.add_logger("logger_copy_test_1", [{"handler": "handler_copy_test_1", "level": "DEBUG"}])
lm.add_logger("logger_copy_test_2", [{"handler": "handler_copy_test_2", "level": "DEBUG"}])

logger_1 = lm.get_logger("logger_copy_test_1")
logger_2 = lm.get_logger("logger_copy_test_2")

 ⚠️ The format referenced by handler 'handler_copy_test_1' is not defined in the 'formats' section of the config file. Using the format as is: 
	 {time} {level} {message} 

 ⚠️ The format referenced by handler 'handler_copy_test_2' is not defined in the 'formats' section of the config file. Using the format as is: 
	 {level} {message} 



#### With `preserve_structure=False`

In [52]:
logger_1.info(f"Testing copy for logger 1 - NOT preserving structure")
logger_2.info(f"Testing copy for logger 2 - NOT preserving structure")

In [53]:
lm.start_copy(
    copy_name = "test_copy",
    path_patterns=[f"./logs/**"],
    copy_destination="./logs_copied",
    copy_interval=5,
    create_dest_dirs=True,
    preserve_structure=False,
    max_retries=3,
    retry_delay=5,
)

Copy worker 'test_copy' started.Started copy operation 'test_copy' with 5s interval.


Copy 'test_copy' found 4 files to copy.


Successfully copied ./logs\copy_test_1.log -> ./logs_copied/copy_test_1.log
Successfully copied ./logs\test.log -> ./logs_copied/test.log
Successfully copied ./logs\subfolder\test2.log -> ./logs_copied/test2.log
Successfully copied ./logs\copy_test_2.log -> ./logs_copied/copy_test_2.log
Copy completed: 4 successful, 0 failed
Copy 'test_copy' found 4 files to copy.
Successfully copied ./logs\copy_test_1.log -> ./logs_copied/copy_test_1.log
Successfully copied ./logs\test.log -> ./logs_copied/test.log
Successfully copied ./logs\subfolder\test2.log -> ./logs_copied/test2.log
Successfully copied ./logs\copy_test_2.log -> ./logs_copied/copy_test_2.log
Copy completed: 4 successful, 0 failed
Copy 'test_copy' found 4 files to copy.
Successfully copied ./logs\copy_test_1.log -> ./logs_copied/copy_test_1.log
Successfully copied ./logs\test.log -> ./logs_copied/test.log
Successfully copied ./logs\subfolder\test2.log -> ./logs_copied/test2.log
Successfully copied ./logs\copy_test_2.log -> ./logs_c

An error will be raised if the copy_name already exists

In [ ]:
lm.start_copy(
    copy_name = "test_copy",
    path_patterns=[f"./logs/**"],
    copy_destination="./logs_copied",
    copy_interval=5,
    create_dest_dirs=True,
    preserve_structure=False,
    max_retries=3,
    retry_delay=5,
)

In [57]:
lm.stop_copy(
    copy_name = "test_copy",
    timeout=60
)

Stopped copy operation 'test_copy'


True

#### With `preserve_structure=True`

In [55]:
logger_1.info(f"Testing copy for logger 1 - preserving structure")
logger_2.info(f"Testing copy for logger 2 - preserving structure")

In [ ]:
root_dir = "./logs"

lm.start_copy(
    copy_name = "test_copy_preserve",
    path_patterns=[f"{root_dir}/**"],
    copy_destination="./logs_copied_preserved",
    root_dir=root_dir,
    copy_interval=5,
    create_dest_dirs=True,
    preserve_structure=True,
    max_retries=3,
    retry_delay=5,
)

Copy worker 'test_copy' started.Started copy operation 'test_copy' with 5s interval.


Copy 'test_copy' found 4 files to copy.


Successfully copied ./logs\copy_test_1.log -> ./logs_copied_preserved/copy_test_1.log
Successfully copied ./logs\test.log -> ./logs_copied_preserved/test.log
Successfully copied ./logs\subfolder\test2.log -> ./logs_copied_preserved/subfolder/test2.log
Successfully copied ./logs\copy_test_2.log -> ./logs_copied_preserved/copy_test_2.log
Copy completed: 4 successful, 0 failed
Copy 'test_copy' found 4 files to copy.
Successfully copied ./logs\copy_test_1.log -> ./logs_copied_preserved/copy_test_1.log
Successfully copied ./logs\test.log -> ./logs_copied_preserved/test.log
Successfully copied ./logs\subfolder\test2.log -> ./logs_copied_preserved/subfolder/test2.log
Successfully copied ./logs\copy_test_2.log -> ./logs_copied_preserved/copy_test_2.log
Copy completed: 4 successful, 0 failed
Copy worker 'test_copy' stopped


#### Other methods/functions

List active copy operations

In [58]:
lm.list_copy_operations()

[]

Manually trigger a copy operation (this will be called atexit too)

In [60]:
# this will trigger a copy for all current active copy operations
lm.trigger_copy_now()

No active copy operations to trigger.


In [ ]:
# this will trigger a  copy operation only for the list of specific copy operations
lm.trigger_copy_now(copy_names=["test_copy_preserve"])
# lm.trigger_copy_now(copy_names=["test_copy", "test_copy_preserve"])

#### Stop copy

In [ ]:
# stop a specific operation
lm.stop_copy(copy_name="test_copy_preserve")

In [ ]:
root_dir = "./logs"

lm.start_copy(
    copy_name = "test_copy_preserved",
    path_patterns=[f"{root_dir}/**"],
    copy_destination="./logs_copied_preserved",
    root_dir=root_dir,
    copy_interval=5,
    create_dest_dirs=True,
    preserve_structure=True,
    max_retries=3,
    retry_delay=5,
)

In [ ]:
# stop all existing copy operations
lm.stop_all_copy()

### Start copy operations from config file

In [40]:
config_file = "./logger_config.yaml"
lm = LogManager(config_file)

Copy enabled (default behavior)
Signal handlers registered


In [41]:
lm.add_handler(
    "handler_file_2",
    {
        "sink": "./logs/subfolder/test2.log",
        "format": "simple",
        "level": "debug"
    }
)

In [42]:
lm.add_logger("logger_c", [{"handler": "handler_file_2", "level": "INFO"}])

In [43]:
logger_A = lm.get_logger("logger_a")
logger_B = lm.get_logger("logger_b")
logger_C = lm.get_logger("logger_c")

In [44]:
logger_A.critical("test logger A")
logger_B.critical("test logger B")
logger_C.critical("test logger C")

 2025-09-03 21:58:41 | CRITICAL | logger_a | 429296937.py     | <module> : 1 - test logger A
 2025-09-03 21:58:41 | CRITICAL | logger_b | 429296937.py     | <module> : 2 - test logger B


In [ ]:
lm.start_copy_from_config()

No copy_config provided, reading from config path.
Copy worker 'test_copy_1' started.
Started copy operation 'test_copy_1' with 5s interval.

Copy worker 'test_copy_2' started.
Started copy operation 'test_copy_2' with 5s interval.

Copy 'test_copy_2' found 1 files to copy.
Copy 'test_copy_1' found 2 files to copy.
Successfully copied ./logs/test.log -> ./logs_copy_2/test.log
Copy completed: 1 successful, 0 failed


Successfully copied ./logs\test.log -> ./logs_copy_1/test.log
Successfully copied ./logs\subfolder\test2.log -> ./logs_copy_1/subfolder/test2.log
Copy completed: 2 successful, 0 failed
Copy worker 'test_copy_1' stopped
Copy worker 'test_copy_2' stopped


In [46]:
lm.stop_all_copy()

Stopped copy operation 'test_copy_1'
Stopped copy operation 'test_copy_2'


[]

### Coordinator

In [61]:
# check if copy is enabled
lm.copy_enabled

True

In [62]:
# get copy status and environment information
lm.get_copy_status()

{'copy_enabled': True,
 'reason': 'Default behavior (enabled)',
 'environment_variable': {'DISABLE_COPY': ''}}

Set `DISABLE_COPY=TRUE` to disable copying

This variable is read only during instantiation of the LogManager

Changes to this variable after LogManager instantiation will not be registered

In [63]:
import os
os.environ["DISABLE_COPY"] = "TRUE"
lm = LogManager()

Copy disabled via DISABLE_COPY environment variable
Config file not provided, initializing logger with class default config.
Signal handlers registered


In [64]:
lm.copy_enabled

False

In [65]:
lm.get_copy_status()

{'copy_enabled': False,
 'reason': 'DISABLE_COPY=true',
 'environment_variable': {'DISABLE_COPY': 'TRUE'}}